In [5]:
import re
import os
from fractions import Fraction
from src.generators.preprocessing import check_key_and_get_slots, fastRandomSampler
from itertools import product


bspacenames = {1: 'singleindep3',
               2: 'doubleindep6',
               3: 'tripleindep12',
               4: 'quadindep24',
               5: 'quintindep45',
               6: 'hexindep85',
               7: 'heptindep155',
               8: 'octindep279'}

brelnames = {1: 'singlerels3',
             2: 'doublerels12',
             3: 'triplerels24',
             4: 'quadrels48',
             5: 'quintrels99',
             6: 'hexrels185',
             7: 'heptrels355',
             8: 'octrels651'}

fspacenames = {1: 'isingleindep3',
               2: 'idoubleindep9',
               3: 'itripleindep21'}

frelnames = {1: 'isinglerels3',
             2: 'idoublerels9',
             3: 'itriplerels33'}



def SB_to_dict(mystring):
    def to_coef(mystr):
        if mystr == '-': return -1
        elif mystr == '': return 1
        else: return int(mystr)
    m=mystring.replace('-','+-').replace(",","").split('+')
    m2=[el.replace("(","").replace(")","").replace("*","").split("SB") for el in m]
    sbdict={elem[1]:to_coef(elem[0]) for elem in m2 if len(elem) > 1}
    return sbdict

def readFBspace(f, prefix):
    res = ''
    reading_form = False
    for line in f:
        if not reading_form:
            if not line.startswith(prefix): continue
            res = ''
            reading_form = True
        if line.isspace(): break
        res += line[:-2] if line[-2] == '\\' else line[:-1]
        if line[-2] in [":", ";"]:
            break
    return res

def get_perm_fspace(fbspacedir,w):
    assert os.path.isfile(f'{fbspacedir}/frontbackspaces/frontspace')
    with open(f'{fbspacedir}/frontbackspaces/frontspace', 'rt') as f:
        mystr=''.join(str.split(readFBspace(f, f'frontspace[{w}]')))
        newstr=re.split(":=|\[|\]",mystr)[4]
        dev = [elem+")" if elem[-1]!=")" else elem for elem in newstr.split("),") if elem]
        basedict= {f'Fp_{w}_{i}':SB_to_dict(el) for i,el in enumerate(dev)}
        flipdict= {}
        for elem,elemdict in basedict.items():
            for term,coef in elemdict.items():
                if term not in flipdict: flipdict[term]={}
                flipdict[term][elem]=basedict[elem][term]
        return basedict,flipdict

def get_perm_bspace(fbspacedir,w):
    assert os.path.isfile(f'{fbspacedir}/frontbackspaces/backspace')
    with open(f'{fbspacedir}/frontbackspaces/backspace', 'rt') as f:
        mystr=''.join(str.split(readFBspace(f, f'backspace[{w}]')))
        newstr=re.split(":=|\[|\]",mystr)[4]
        dev = [elem+")" if elem[-1]!=")" else elem for elem in newstr.split("),") if elem]
        basedict= {f'Bp_{w}_{i}':SB_to_dict(el) for i,el in enumerate(dev)}
        flipdict= {}
        for elem,elemdict in basedict.items():
            for term,coef in elemdict.items():
                if term not in flipdict: flipdict[term]={}
                flipdict[term][elem]=basedict[elem][term]
        return basedict,flipdict

def get_rest_bspace(fbspacedir, w):
    assert os.path.isfile(f'{fbspacedir}/multifinal_new_norm')
    with open(f'{fbspacedir}/multifinal_new_norm', 'rt') as f:
        res = readFBspace(f,str(bspacenames[w]))
        myset={elem for elem in re.split(":=\[|E\(|\)|\]:", re.sub('[, *]', '', res))[1:] if elem}
        myd = {elem:f'Br_{w}_{i}' for i, elem in enumerate(myset)}
        flip = {f'Br_{w}_{i}':elem for i, elem in enumerate(myset)}
        return myd, flip
    
def get_rest_fspace(fbspacedir, w):
    assert os.path.isfile(f'{fbspacedir}/ClipFrontTriple')
    with open(f'{fbspacedir}/ClipFrontTriple', 'rt') as f:
        res = readFBspace(f,str(fspacenames[w]))
        myset={elem for elem in re.split(":=\[|SB\(|\)|\]:", re.sub('[, *]', '', res))[1:] if elem}
        myd = {elem:f'Fr_{w}_{i}' for i, elem in enumerate(myset)}
        flip = {f'Fr_{w}_{i}':elem for i, elem in enumerate(myset)}
        return myd, flip
    
def getBrel_eqs(f, w):
    res = ''
    reading_form = False
    res = readFBspace(f,str(brelnames[w]))
    out = [re.sub('\s+', '', elem) for elem in re.split(":= \[|,|\] :", re.sub(',\s*(?=[^()]*\))', '', res))[1:]]
    return out

def getFrel_eqs(f, w):
    res = ''
    reading_form = False
    res = readFBspace(f,str(frelnames[w]))
    out = [re.sub('\s+', '', elem) for elem in re.split(":= \[|,|\] :", re.sub(',\s*(?=[^()]*\))', '', res))[1:]]
    return out


def rel_to_dict(relstring,bspace=True):
    # read an F/Bspace rel as a nested dict. if the rel is
    #E(abc)=-2*E(def)+4*E(bcd), return {abc: {def:-2, bcd:4}}
    def expandcoef(c): return Fraction(c + '1') if len(c) == 1 else Fraction(c)
    if bspace:
        newstring = [elem for elem in re.split("=|E\(|\)", re.sub('#', '', re.sub('[,*]', '', relstring))) if elem]
    else:
        newstring = [elem for elem in re.split("=|SB\(|\)", re.sub('#', '', re.sub('[,*]', '', relstring))) if elem]

    if len(newstring) == 0: return {None: None}
    if newstring[1] == '0':
        reldict = {None: 0}
    else:
        if newstring[1].isalpha():
            eq = ['+'] + newstring[1:]
        else:
            eq = newstring[1:]

        if len(eq) == 1:
            reldict = {eq[0]: 1}
        else:
            reldict = {k: expandcoef(c) for i, (c, k) in enumerate(zip(eq, eq[1:])) if i % 2 == 0}

    out = {newstring[0]: reldict}
    return out

def get_brels(relpath):
    with open(f'{relpath}/multifinal_new_norm', 'rt') as f:
        brels={}
        for i in range(2,9):
            brels[i]={k:v for j in getBrel_eqs(f,i) for k,v in rel_to_dict(j).items() if k}
        return brels

def get_frels(relpath):
    with open(f'{relpath}/ClipFrontTriple', 'rt') as f:
        brels={}
        for i in range(1,4):
            brels[i]={k:v for j in getFrel_eqs(f,i) for k,v in rel_to_dict(j,False).items() if k}
        return brels
    
def all_perm_bspaces(relpath):
    perm_bspace, perm_bspace_keysfirst={},{}
    for i in range(2,9):
        perm_bspace |= get_perm_bspace(relpath,i)
        

ModuleNotFoundError: No module named 'src'

In [6]:
def get_rels_perm(mydir, weight, seam="front",reltype="oneletter"):
    #get the permissive rels pre-generated from seam_sewing_rels.py. 
    #These are the checks generated by Lance & Garrett (lettbackrels and lettfrontrels).
    
    if not reltype in ["oneletter","twoletter"]: return
    r=reltype
    
    if seam == "back":
        file=f"Bspace_rels_{r}"
        prefix="sewrelsb"
    elif seam == "front":
        file=f"Fspace_rels_{r}"
        prefix="sewrelsf"
    else:
        print("bad seam type!")
        raise ValueError

    assert os.path.isfile(f'{mydir}/{file}')
    with open(f'{mydir}/{file}', 'rt') as f:
        checks=readFBspace(f,f'{prefix}[{weight}]')

    c=[i for i in checks.split("\'")[:-1] if i != ', ' and 'sewrels' not in i]

    return c

def readcrel(crel,w=2,seam="back"):
    def cterm_to_Fp(cterm):
        if not ',' in cterm:
            print("bad term!")
            raise ValueError
        numlet=re.split(",",cterm)
        o=[]
        for i in numlet:
            if i.isnumeric():
                if seam == "front":
                    o.append(f'Fp_{w}_{i}@')
                elif seam == "back":
                    o.append(f'@Bp_{w}_{i}')
                else: raise ValueError
            else: o.append(i)
        return ''.join(o)

    def numstr_to_num(numstr):
        if numstr=='+': return 1
        elif numstr == '-': return -1
        elif '*' in numstr: 
            numstr=numstr.replace('−','-')
            return Fraction(numstr[:-1])
            #else:return int(numstr[:-1])
    
    myrel=re.split("c\[|\]",crel)[:-1]
    coefs = [elem for i,elem in enumerate(myrel) if i%2==0]
    terms = [elem for i,elem in enumerate(myrel) if i%2==1]
    return {cterm_to_Fp(let):numstr_to_num(num) for num,let in zip(coefs,terms)}
        
def get_relpermdict(mydir,w,seam,reltype):
    return [readcrel(i,w,seam) for i in get_rels_perm(mydir,w,seam,reltype)]    

In [7]:
str.split(']')

[']']

In [8]:
get_rest_bspace('/home/garrett/LanceData_New',8)[1]

NameError: name 'get_rest_bspace' is not defined

In [9]:
relpath='/home/garrett/LanceData_New/'
brels=get_brels(relpath)
frels=get_frels(relpath)

rfspaces={w:get_rest_fspace(relpath,w)[1] for w in range(1,4)}
rbspaces={w:get_rest_bspace(relpath,w)[1] for w in range(1,8)}
pfspaces={w:get_perm_fspace(relpath,w)[1] for w in range(1,7)}
pbspaces={w:get_perm_bspace(relpath,w)[1] for w in range(1,8)}

rfspaces_flip={w:get_rest_fspace(relpath,w)[0] for w in range(1,4)}
rbspaces_flip={w:get_rest_bspace(relpath,w)[0] for w in range(1,8)}
pfspaces_flip={w:get_perm_fspace(relpath,w)[0] for w in range(1,7)}
pbspaces_flip={w:get_perm_bspace(relpath,w)[0] for w in range(1,8)}

pb_1l_sewchecks={w:get_relpermdict(relpath,w,"back","oneletter") for w in range(1,8)}
pb_2l_sewchecks={w:get_relpermdict(relpath,w,"back","twoletter") for w in range(1,8)}
pf_1l_sewchecks={w:get_relpermdict(relpath,w,"front","oneletter") for w in range(1,7)}
pf_2l_sewchecks={w:get_relpermdict(relpath,w,"front","twoletter") for w in range(1,7)}

NameError: name 'get_brels' is not defined

In [558]:

                #if not elem: continue
                #else: brels[i] |= elem

In [559]:
rfspaces_flip[3]

{'aff': 'Fr_3_0',
 'aae': 'Fr_3_1',
 'bbd': 'Fr_3_2',
 'ccd': 'Fr_3_3',
 'cee': 'Fr_3_4',
 'ccb': 'Fr_3_5',
 'afa': 'Fr_3_6',
 'bdd': 'Fr_3_7',
 'bdb': 'Fr_3_8',
 'cec': 'Fr_3_9',
 'bba': 'Fr_3_10',
 'cca': 'Fr_3_11',
 'aac': 'Fr_3_12',
 'aab': 'Fr_3_13',
 'bbf': 'Fr_3_14',
 'ccc': 'Fr_3_15',
 'aaf': 'Fr_3_16',
 'bbb': 'Fr_3_17',
 'bbc': 'Fr_3_18',
 'cce': 'Fr_3_19',
 'aaa': 'Fr_3_20'}

In [560]:
def get_image(word, rownum):
    return ''.join([dihedral_table[rownum][idx] for idx in [alphabet.index(l) for l in [*word]]])

def table_to_rels(table):
    tr=[{get_image(k,ind):v for k,v in rel.items()} for ind in range(len(alphabet)) for rel in table]
    return [i for n, i in enumerate(tr) if not 
               any(set(sorted(i.keys())) == set(sorted(k.keys())) for k in tr[n + 1:])]





# triple-adjacency relation: plus dihedral symmetry; any slot
# integrability relations: any slot

alphabet=['a','b','c','d','e','f']
from itertools import permutations

dihedral_table = [list(permutations(alphabet[:3]))[i]+list(permutations(alphabet[3:]))[i]
                  for i in range(len(alphabet))]
triple_table = [{'aab': 1, 'abb': 1, 'acb': 1}]
pair_table = [{'ab': 1, 'ac': 1, 'ba': -1, 'ca': -1},  # eq 3.6
                              {'ca': 1, 'cb': 1, 'ac': -1, 'bc': -1},  # eq 3.7
                              {'db': 1, 'dc': -1, 'bd': -1, 'cd': 1, 'ec': 1, 'ea': -1, 'ce': -1,
                               'ae': 1, 'fa': 1, 'fb': -1, 'af': -1, 'bf': 1, 'cb': 2,'bc': -2},
                                 {'ad':1},{'da':1},{'df':1}]

pair_rels=table_to_rels(pair_table)
triple_rels=table_to_rels(triple_table)

In [561]:
rfspaces_flip[2]

{'aa': 'Fr_2_0',
 'bc': 'Fr_2_1',
 'af': 'Fr_2_2',
 'bb': 'Fr_2_3',
 'ce': 'Fr_2_4',
 'ca': 'Fr_2_5',
 'bd': 'Fr_2_6',
 'cc': 'Fr_2_7',
 'ab': 'Fr_2_8'}

In [562]:
get_all_checks(3,3,2,"PR")
    

getting back pair rels
{'d@Br_2_1': 0}
{'a@Br_2_1': 0, 'a@Br_2_5': 0, 'b@Br_2_1': 0, 'b@Br_2_2': 0, 'b@Br_2_4': 0, 'c@Br_2_3': 0, 'c@Br_2_4': 0, 'c@Br_2_5': 0, 'd@Br_2_2': 0, 'd@Br_2_3': 0, 'e@Br_2_0': 0, 'e@Br_2_2': 0, 'f@Br_2_0': 0, 'f@Br_2_3': 0}
{'a@Br_2_4': 0}
{'d@Br_2_0': 0}
{'d@Br_2_5': 0}
{'e@Br_2_1': 0}
{'a@Br_2_1': 0, 'a@Br_2_2': 0, 'a@Br_2_5': 0, 'b@Br_2_1': 0, 'b@Br_2_4': 0, 'c@Br_2_0': 0, 'c@Br_2_4': 0, 'c@Br_2_5': 0, 'd@Br_2_2': 0, 'd@Br_2_3': 0, 'e@Br_2_0': 0, 'e@Br_2_2': 0, 'f@Br_2_0': 0, 'f@Br_2_3': 0}
{'b@Br_2_5': 0}
{'e@Br_2_3': 0}
{'e@Br_2_4': 0}
{'a@Br_2_3': 0, 'b@Br_2_0': 0, 'b@Br_2_2': 0, 'c@Br_2_3': 0}
{'f@Br_2_5': 0}
{'a@Br_2_2': 0, 'b@Br_2_2': 0, 'c@Br_2_0': 0, 'c@Br_2_3': 0}
{'a@Br_2_2': 0, 'a@Br_2_3': 0, 'b@Br_2_0': 0, 'c@Br_2_0': 0}
{'a@Br_2_1': 0, 'a@Br_2_3': 0, 'a@Br_2_5': 0, 'b@Br_2_0': 0, 'b@Br_2_1': 0, 'b@Br_2_4': 0, 'c@Br_2_4': 0, 'c@Br_2_5': 0, 'd@Br_2_2': 0, 'd@Br_2_3': 0, 'e@Br_2_0': 0, 'e@Br_2_2': 0, 'f@Br_2_0': 0, 'f@Br_2_3': 0}
{'c@Br_2_1': 0}
{

[{'d@Br_2_1': 1},
 {'d@Br_2_2': 1,
  'd@Br_2_3': -1,
  'c@Br_2_4': -1,
  'b@Br_2_4': 1,
  'f@Br_2_3': 1,
  'e@Br_2_2': -1,
  'b@Br_2_2': 2,
  'c@Br_2_3': -2},
 {'d@Br_2_3': -1,
  'f@Br_2_3': 1,
  'f@Br_2_0': -1,
  'b@Br_2_1': -1,
  'a@Br_2_1': 1,
  'e@Br_2_0': 1,
  'c@Br_2_3': -2},
 {'d@Br_2_2': 1,
  'f@Br_2_0': -1,
  'e@Br_2_0': 1,
  'e@Br_2_2': -1,
  'a@Br_2_5': -1,
  'c@Br_2_5': 1,
  'b@Br_2_2': 2},
 {'a@Br_2_4': 1},
 {'d@Br_2_0': 1},
 {'d@Br_2_5': 1},
 {'e@Br_2_1': 1},
 {'e@Br_2_2': 1,
  'e@Br_2_0': -1,
  'c@Br_2_5': -1,
  'a@Br_2_5': 1,
  'f@Br_2_0': 1,
  'd@Br_2_2': -1,
  'a@Br_2_2': 2,
  'c@Br_2_0': -2},
 {'e@Br_2_0': -1,
  'f@Br_2_0': 1,
  'f@Br_2_3': -1,
  'a@Br_2_1': -1,
  'b@Br_2_1': 1,
  'd@Br_2_3': 1,
  'c@Br_2_0': -2},
 {'e@Br_2_2': 1,
  'f@Br_2_3': -1,
  'd@Br_2_3': 1,
  'd@Br_2_2': -1,
  'b@Br_2_4': -1,
  'c@Br_2_4': 1,
  'a@Br_2_2': 2},
 {'b@Br_2_5': 1},
 {'e@Br_2_3': 1},
 {'e@Br_2_4': 1},
 {'b@Br_2_2': 1, 'c@Br_2_3': -1, 'a@Br_2_3': -1},
 {'b@Br_2_0': 1, 'c@Br_2_3': -

In [672]:
from collections import defaultdict

#given: a seam letter and indep bspace elem
#AND: a nonlocal relation (integ, triple, etc.)
#get: the other non-indep terms it is related to by this relation
#then: given a non-indep term, get the indeps it is related to by bspace/fspace rels

#example:
#first, do: a*B[1,3]= 1*a*(bcd) - 1*a*(bce) by integ3
#next, do: a*(bcd)= 1*a*B[2,3], etc. by back triples
#return: {rel instance over letter+indeps that encodes the nonlocal}
#for "LettbackRels", we have this already in permissive format!

def possible_slots(fweight,seamsize,bweight):
    tot=fweight+seamsize+bweight
    if tot % 2 != 0:
        print("not a valid sew matrix!")
        return
    
    slots={}
    slots['pair']={'front':{fweight-1},
                   'back':{tot-bweight-1}}
    slots['triple']={'front':{fweight-2,fweight-1} if fweight > 1 else {fweight-1},
                    'back':{tot-bweight-2,tot-bweight-1} if bweight > 1 else {tot-bweight-2}}
    
    if seamsize > 1:
        slots['pair']['seam']={k for k in range(fweight,tot-bweight-1)}
    if seamsize > 2:
        slots['triple']['seam']={k for k in range(fweight,tot-bweight-2)}
        
    return slots

def expand_rform_elem(key, opt="all"):
    def proc_elem(myelem):
        for entry in myelem.split('@'):
            if entry.isalpha(): 
                for i in entry:
                    yield i
            else: yield entry
        
    elem=[out for out in proc_elem(key)]
    if opt == "front":
        fweight,bweight=int(elem[0].split('_')[1]),None
        expanded=[i for i in rfspaces[fweight][elem[0]]]+elem[1:]
    elif opt == "back":
        fweight,bweight=None,int(elem[-1].split('_')[1])
        expanded=elem[:-1]+[i for i in rbspaces[bweight][elem[-1]]]
    elif opt == "all":
        fweight,bweight=int(elem[0].split('_')[1]),int(elem[-1].split('_')[1])
        expanded=[i for i in rfspaces[fweight][elem[0]]]+elem[1:-1]+[i for i in rbspaces[bweight][elem[-1]]]
    else:
        raise ValueError
    return expanded, (fweight,bweight)

def compress_rform_elem(key,fweight=None,bweight=None,seam=None):
    if fweight:
        fseam=fweight
        myf=[rfspaces_flip[fweight][''.join(key[:fseam])]]
    if bweight:
        bseam=len(key)-bweight
        myb=[rbspaces_flip[bweight][''.join(key[bseam:])]]
        
    if seam =="front":
        l=myf+[''.join(key[fseam:])]
    elif seam =="back":
        l=[''.join(key[:bseam])]+myb
    elif seam =="all":
        l=myf+[''.join(key[fseam:bseam])]+myb
    return '@'.join(l)
         

def apply_rel(myelem,rel,slot,seam,rel_len):
    #assume key looks like [f_elem,l1,l2,..,ln,b_elem]
    #assume slot is w.r.t the UNcompressed key.
    #last arg: which seam to apply rel at (front or back?)
    fullkey, (fweight, bweight) = expand_rform_elem(myelem,seam)
    #if not ''.join(fullkey[slot:slot+rel_len]) in rel:
    #    print(rel)
    
    pair = True if rel_len == 2 else False
    triple= True if rel_len == 3 else False
    if seam == "back": 
        weight=bweight
        if not ((pair and slot == (len(fullkey) - weight -1)) or 
                (triple and slot == (len(fullkey) - weight -2)) or (triple and slot == (len(fullkey) - weight -1))):
            print("Error, bad slot!")
            raise ValueError
    elif seam == "front":
        weight=fweight
        if not ((pair and slot == (weight-1)) or 
                (triple and slot == (weight-2)) or (triple and slot == (weight-1))):
            print("Error, bad slot!")
            raise ValueError
    elif seam == "seam":
        if len(fullkey)-fweight-bweight < rel_len:
            print("Error, bad rel!")
            raise ValueError

        if not (slot > fweight-1 and slot < len(fullkey)-bweight-1):
            print("Error, bad slot!")
            raise ValueError    
    else:
        print("Specify front, back, or seam!")
        raise ValueError

    
    #given: an indep term, get: the non-indeps it is related to by the given rel.
    non_indeps=get_related(rel,fullkey,slot)
    result=defaultdict(int)    
    #given: a list of non-indeps, get the indeps they are related to:
    for related,coef in non_indeps.items():
        if seam == "seam":
            c=compress_rform_elem(related.split(','),fweight,bweight,"all")
            result[c] += int(coef)
        else:
            mydic= get_as_indepsum(related.split(','),fweight,bweight,seam)
            for k, v in mydic.items():
                result[k] += int(coef*v) 
    out=dict(result)
    return out

def apply_rel_in_seam(rel,fullkey,slot):
    #given: a string in exposed seam
    non_indeps=get_related(rel,fullkey,slot)
    result=defaultdict(int)    
    #given: a list of non-indeps, get the indeps they are related to:
    for related,coef in non_indeps.items():
        c=''.join(related.split(','))
        result[c] += int(coef)
    out=dict(result)
    return out

def generate(fastsymb,rel,sp,rel_len,slot):
    while len(fastsymb) > 0:
        seedkey=fastsymb.random_key()
        inst=apply_rel(seedkey,rel,slot,sp,rel_len)
        isbad=False
        for k in inst:
            if k not in fastsymb: isbad=True
            else:fastsymb.remove(k)
        if not isbad:
            yield inst  

def gen_with_slots(fastsymb,rel,sp,rel_len):
    while len(fastsymb) > 0:
        seedkey=fastsymb.random_key()
        slot=fastsymb[seedkey].random_key()
        inst=apply_rel_in_seam(rel,seedkey,slot)
        isbad=False
        for k in inst:
            if k not in fastsymb:
                isbad=True
            else:
                if slot not in fastsymb[k]:
                    isbad=True
                else:
                    fastsymb[k].remove(slot)
                    if len(fastsymb[k]) == 0:
                        fastsymb.remove(k)
        if not isbad:
            yield inst 
            
def get_all_rest_rels(fweight,seamsize,bweight):    
    fkeys=rfspaces[fweight].keys()
    allseams= [''.join(i) for i in product(alphabet, repeat=seamsize)]
    bkeys=rbspaces[bweight].keys()

    #get the sewing matrix and the possible slots
    allsewkeys={'@'.join([f,m,b]) for f in fkeys for m in allseams for b in bkeys}
    pslots=possible_slots(fweight,seamsize,bweight)
    spaces=['front','back','seam']
    all_rels=[]
    for sp in spaces:
        print(f"getting {sp} pair rels")
        for rel in pair_rels:
            fastsymb=fastRandomSampler({k:slotset for k in allsewkeys 
                      if len(slotset := fastRandomSampler({slot for slot in pslots['pair'][sp] if 
                                         ''.join(expand_rform_elem(k)[0][slot:slot+2]) in rel})) > 0})
            all_rels += [inst for inst in generate(fastsymb,rel,sp,2)]
        print(f"getting {sp} triple rels")
        for rel in triple_rels:
            fastsymb=fastRandomSampler({k:slotset for k in allsewkeys 
                      if len(slotset := fastRandomSampler({slot for slot in pslots['triple'][sp] if 
                                         ''.join(expand_rform_elem(k)[0][slot:slot+3]) in rel})) > 0})
            all_rels += [inst for inst in generate(fastsymb,rel,sp,3)]

    return all_rels

def get_all_checks(fweight,seamsize,bweight,pattern=None):    
    totlen=fweight+seamsize+bweight
    def isgood(myseam):
        bads={'ad','be','cf','da','eb','fc','df','fd','ef','fe','de','ed'}
        for i,j in zip(myseam[:-1],myseam[1:]):
            if ''.join([i,j]) in bads: return False
        return True
    
    def allseams(s):
        return [''.join(i) for i in product(alphabet, repeat=s) if isgood(i)]

    if not pattern in ["RR","PR","PP","RP"]:
        print("Error! bad sewmat configuration!")
        raise ValueError
    
    #get the sewing matrix and the possible slots

    if pattern in ["RR","RP"]:
        fkeys=rfspaces[fweight].keys()
    else:
        fkeys=pfspaces_flip[fweight].keys()    
        
    if pattern in ["RR","PR"]:
        bkeys=rbspaces[bweight].keys()
    else:
        bkeys=pbspaces_flip[bweight].keys()
    
    rspaces,pspaces,pslots=[],[],{'pair':{},'triple':{}}
    if pattern in ["RR","RP"]:rspaces.append("front")
    if pattern in ["RR","PR"]:rspaces.append("back")
    if pattern in ["PP","PR"]:pspaces.append("front")
    if pattern in ["PP","RP"]:pspaces.append("back")
    all_rels=[]
    
    pslots['pair']['front']=[fweight-1]
    pslots['triple']['front']=[fweight-2,fweight-1] if fweight > 1 else [fweight-1]
    pslots['pair']['back']=[seamsize]
    pslots['triple']['back']=[seamsize-1,seamsize] if seamsize > 1 else [seamsize]
    pslots['pair']['seam']=[k for k in range(0,seamsize-1)]
    pslots['triple']['seam']=[k for k in range(0,seamsize-2)]
    
    all_rels={}
    if seamsize > 0:
        all_rels['b_1']=[]
        all_rels['f_1']=[]
    if seamsize > 1:
        all_rels['b_2']=[]
        all_rels['f_2']=[]
    if seamsize > 1:
        all_rels['seam']=[]
        
    #expand the relevant f/b elem
    for sp in rspaces:
        print(f"getting {sp} pair rels")
        for rel in pair_rels:
            if sp == "back":
                fastsymb=fastRandomSampler({'@'.join([let,b]):0 for let in alphabet
                                            for b,blet in rbspaces[bweight].items() if ''.join([let,blet[0]]) in rel})          
                all_rels['b_1']+=[inst for inst in generate(fastsymb,rel,sp,2,0)]
            
            if sp == "front":
                fastsymb=fastRandomSampler({'@'.join([f,let]):0 for let in alphabet
                                        for f,flet in rfspaces[fweight].items() if ''.join([flet[-1],let]) in rel})
                all_rels['f_1']+= [inst for inst in generate(fastsymb,rel,sp,2,fweight-1)]
              
            
        print(f"getting {sp} triple rels")
        for rel in triple_rels:
            if sp == "front":
                fastsymb=fastRandomSampler({'@'.join([f,let]):0 for let in alphabet
                                            for f,flet in rfspaces[fweight].items() if ''.join([f[:-2],let]) in rel})
                
                all_rels['f_1'] += [inst for inst in generate(fastsymb,rel,sp,3,fweight-1)]
                if seamsize > 1:
                    fastsymb=fastRandomSampler({'@'.join([f,let]):0 for let in allseams(2)
                            for f,flet in rfspaces[fweight].items() if ''.join([f[-1],let]) in rel})
                all_rels['f_2'] += [inst for inst in generate(fastsymb,rel,sp,3,fweight-2)]
    
                
            if sp == "back":
                fastsymb=fastRandomSampler({'@'.join([let,b]):0 for let in alphabet
                                            for b,blet in rbspaces[bweight].items() if ''.join([let,blet[:2]]) in rel})                
                all_rels['b_1'] += [inst for inst in generate(fastsymb,rel,sp,3,0)]
                if seamsize > 1:
                    fastsymb=fastRandomSampler({'@'.join([let,b]):0 for let in allseams(2)
                                               for b,blet in rbspaces[bweight].items() if ''.join([let,blet[0]]) in rel})
                    all_rels['b_2'] += [inst for inst in generate(fastsymb,rel,sp,3,0)]

    #perm
    for sp in pspaces:
        print(f"getting {sp} permissive rels")
        if sp == "front":
            all_rels['f_1'] += pf_1l_sewchecks[fweight]
            if seamsize > 2:
                all_rels['f_2'] +=pf_2l_sewchecks[fweight]
        if sp == "back":
            all_rels['b_1'] += pb_1l_sewchecks[fweight]
            if seamsize > 2:
                all_rels['b_2'] += pb_2l_sewchecks[fweight]
    
    print("getting rels inside seam")
    for rel in pair_rels:
        fastsymb=fastRandomSampler({k:slotset for k in allseams(seamsize) 
                  if len(slotset := fastRandomSampler({slot for slot in pslots['pair']['seam'] if 
                                     ''.join(k[slot:slot+2]) in rel})) > 0})
        all_rels['seam'] += [inst for inst in gen_with_slots(fastsymb,rel,sp,2)]
   
    for rel in triple_rels:
        fastsymb=fastRandomSampler({k:slotset for k in allseams(seamsize) 
                  if len(slotset := fastRandomSampler({slot for slot in pslots['triple']['seam'] if 
                                     ''.join(k[slot:slot+3]) in rel})) > 0})
        all_rels['seam'] += [inst for inst in gen_with_slots(fastsymb,rel,sp,3)]

    flist1=['@'.join(i) for i in product(allseams(seamsize-1),bkeys)]
    flist2=['@'.join(i) for i in product(allseams(seamsize-2),bkeys)]
    blist1=['@'.join(i) for i in product(fkeys,allseams(seamsize-1))]
    blist2=['@'.join(i) for i in product(fkeys,allseams(seamsize-2))]
    edges=[i for i in product(fkeys,bkeys)]

    print("in aggregated 'c[i,a]' form, there are:")
    print(f"{len(all_rels['f_2'])+len(all_rels['f_1'])} checks across front seam")
    print(f"{len(all_rels['b_2'])+len(all_rels['b_1'])} checks across back seam")
    print(f"{len(all_rels['seam'])} checks inside seam")
    print("Gluing:")
    
    tot_rels=[]
    tot_rels += [{k+elem:v for k,v in rel.items()} for elem in flist1 for rel in all_rels['f_1']]
    tot_rels += [{k+elem:v for k,v in rel.items()} for elem in flist2 for rel in all_rels['f_2']]  
    tot_rels += [{elem+k:v for k,v in rel.items()} for elem in blist1 for rel in all_rels['b_1']]
    tot_rels += [{elem+k:v for k,v in rel.items()} for elem in blist2 for rel in all_rels['b_2']]
    tot_rels += [{'@'.join([e[0],k,e[1]]):v for k,v in rel.items()} for e in edges for rel in all_rels['seam']]
 
    print(f'there are {len(tot_rels)} checks!')
    return tot_rels


In [563]:
from src.generators.rels_utils_refactored import fastRandomSampler,get_coeff_from_word,check_slot,find_all,alphabet,count_appearances



In [659]:
def get_related(rel,seed,slot):
    seedword=list(seed)
    return {','.join(seedword[:slot]+[i for i in elem]+seedword[slot+len(elem):]):relcoef 
            for elem,relcoef in rel.items()}

def get_as_indepsum(fullword,fweight,bweight,seam=None):
    if seam=="back":
        indeps = rbspaces_flip[bweight]
        body,belem=fullword[:-bweight],''.join(fullword[-bweight:])
        if belem in indeps:
            reldict={compress_rform_elem(fullword,fweight,bweight,seam):1}
        else:
            mydict=brels[bweight][belem]
            reldict={compress_rform_elem(body+[i for i in k],fweight,bweight,seam):v
                    for k,v in mydict.items() if k}
    elif seam=="front":
        indeps=rfspaces_flip[fweight]
        felem,body=''.join(fullword[:fweight]), fullword[fweight:]
        if felem in indeps:
            reldict={compress_rform_elem(fullword,fweight,bweight,seam):1}
        else: 
            mydict=frels[fweight][felem]
            reldict={compress_rform_elem([i for i in k]+body,fweight,bweight,seam):v 
                    for k,v in mydict.items() if k}
    else:
        print("Bad seam!")
        raise ValueError
    return(reldict)

from collections import defaultdict

#given: a seam letter and indep bspace elem
#AND: a nonlocal relation (integ, triple, etc.)
#get: the other non-indep terms it is related to by this relation
#then: given a non-indep term, get the indeps it is related to by bspace/fspace rels

#example:
#first, do: a*B[1,3]= 1*a*(bcd) - 1*a*(bce) by integ3
#next, do: a*(bcd)= 1*a*B[2,3], etc. by back triples
#return: {rel instance over letter+indeps that encodes the nonlocal}
#for "LettbackRels", we have this already in permissive format!

def possible_slots(fweight,seamsize,bweight):
    tot=fweight+seamsize+bweight
    if tot % 2 != 0:
        print("not a valid sew matrix!")
        return
    
    slots={}
    slots['pair']={'front':{fweight-1},
                   'back':{tot-bweight-1}}
    slots['triple']={'front':{fweight-2,fweight-1} if fweight > 1 else {fweight-1},
                    'back':{tot-bweight-2,tot-bweight-1} if bweight > 1 else {tot-bweight-2}}
    
    if seamsize > 1:
        slots['pair']['seam']={k for k in range(fweight,tot-bweight-1)}
    if seamsize > 2:
        slots['triple']['seam']={k for k in range(fweight,tot-bweight-2)}
        
    return slots

def expand_rform_elem(key, opt="all"):
    def proc_elem(myelem):
        for entry in myelem.split('@'):
            if entry.isalpha(): 
                for i in entry:
                    yield i
            else: yield entry
        
    elem=[out for out in proc_elem(key)]
    if opt == "front":
        fweight,bweight=int(elem[0].split('_')[1]),None
        expanded=[i for i in rfspaces[fweight][elem[0]]]+elem[1:]
    elif opt == "back":
        fweight,bweight=None,int(elem[-1].split('_')[1])
        expanded=elem[:-1]+[i for i in rbspaces[bweight][elem[-1]]]
    elif opt == "all":
        fweight,bweight=int(elem[0].split('_')[1]),int(elem[-1].split('_')[1])
        expanded=[i for i in rfspaces[fweight][elem[0]]]+elem[1:-1]+[i for i in rbspaces[bweight][elem[-1]]]
    else:
        raise ValueError
    return expanded, (fweight,bweight)

def compress_rform_elem(key,fweight=None,bweight=None,seam=None):
    if fweight:
        fseam=fweight
        myf=[rfspaces_flip[fweight][''.join(key[:fseam])]]
    if bweight:
        bseam=len(key)-bweight
        myb=[rbspaces_flip[bweight][''.join(key[bseam:])]]
        
    if seam =="front":
        l=myf+[''.join(key[fseam:])]
    elif seam =="back":
        l=[''.join(key[:bseam])]+myb
    elif seam =="all":
        l=myf+[''.join(key[fseam:bseam])]+myb
    return '@'.join(l)
         

def apply_rel(myelem,rel,slot,seam,rel_len):
    #assume key looks like [f_elem,l1,l2,..,ln,b_elem]
    #assume slot is w.r.t the UNcompressed key.
    #last arg: which seam to apply rel at (front or back?)
    fullkey, (fweight, bweight) = expand_rform_elem(myelem,seam)
    #if not ''.join(fullkey[slot:slot+rel_len]) in rel:
    #    print(rel)
    
    pair = True if rel_len == 2 else False
    triple= True if rel_len == 3 else False
    if seam == "back": 
        weight=bweight
        if not ((pair and slot == (len(fullkey) - weight -1)) or 
                (triple and slot == (len(fullkey) - weight -2)) or (triple and slot == (len(fullkey) - weight -1))):
            print("Error, bad slot!")
            raise ValueError
    elif seam == "front":
        weight=fweight
        if not ((pair and slot == (weight-1)) or 
                (triple and slot == (weight-2)) or (triple and slot == (weight-1))):
            print("Error, bad slot!")
            raise ValueError
    elif seam == "seam":
        if len(fullkey)-fweight-bweight < rel_len:
            print("Error, bad rel!")
            raise ValueError

        if not (slot > fweight-1 and slot < len(fullkey)-bweight-1):
            print("Error, bad slot!")
            raise ValueError    
    else:
        print("Specify front, back, or seam!")
        raise ValueError

    
    #given: an indep term, get: the non-indeps it is related to by the given rel.
    non_indeps=get_related(rel,fullkey,slot)
    result=defaultdict(int)    
    #given: a list of non-indeps, get the indeps they are related to:
    for related,coef in non_indeps.items():
        if seam == "seam":
            c=compress_rform_elem(related.split(','),fweight,bweight,"all")
            result[c] += int(coef)
        else:
            mydic= get_as_indepsum(related.split(','),fweight,bweight,seam)
            for k, v in mydic.items():
                result[k] += int(coef*v) 
    out=dict(result)
    return out

def apply_rel_in_seam(rel,fullkey,slot):
    #given: a string in exposed seam
    non_indeps=get_related(rel,fullkey,slot)
    result=defaultdict(int)    
    #given: a list of non-indeps, get the indeps they are related to:
    for related,coef in non_indeps.items():
        c=''.join(related.split(','))
        result[c] += int(coef)
    out=dict(result)
    return out

def generate(fastsymb,rel,sp,rel_len,slot):
    while len(fastsymb) > 0:
        seedkey=fastsymb.random_key()
        inst=apply_rel(seedkey,rel,slot,sp,rel_len)
        isbad=False
        for k in inst:
            if k not in fastsymb: isbad=True
            else:fastsymb.remove(k)
        if not isbad:
            yield inst  

def gen_with_slots(fastsymb,rel,sp,rel_len):
    while len(fastsymb) > 0:
        seedkey=fastsymb.random_key()
        slot=fastsymb[seedkey].random_key()
        inst=apply_rel_in_seam(rel,seedkey,slot)
        isbad=False
        for k in inst:
            if k not in fastsymb:
                isbad=True
            else:
                if slot not in fastsymb[k]:
                    isbad=True
                else:
                    fastsymb[k].remove(slot)
                    if len(fastsymb[k]) == 0:
                        fastsymb.remove(k)
        if not isbad:
            yield inst 
            
def get_all_rest_rels(fweight,seamsize,bweight):    
    fkeys=rfspaces[fweight].keys()
    allseams= [''.join(i) for i in product(alphabet, repeat=seamsize)]
    bkeys=rbspaces[bweight].keys()

    #get the sewing matrix and the possible slots
    allsewkeys={'@'.join([f,m,b]) for f in fkeys for m in allseams for b in bkeys}
    pslots=possible_slots(fweight,seamsize,bweight)
    spaces=['front','back','seam']
    all_rels=[]
    for sp in spaces:
        print(f"getting {sp} pair rels")
        for rel in pair_rels:
            fastsymb=fastRandomSampler({k:slotset for k in allsewkeys 
                      if len(slotset := fastRandomSampler({slot for slot in pslots['pair'][sp] if 
                                         ''.join(expand_rform_elem(k)[0][slot:slot+2]) in rel})) > 0})
            all_rels += [inst for inst in generate(fastsymb,rel,sp,2)]
        print(f"getting {sp} triple rels")
        for rel in triple_rels:
            fastsymb=fastRandomSampler({k:slotset for k in allsewkeys 
                      if len(slotset := fastRandomSampler({slot for slot in pslots['triple'][sp] if 
                                         ''.join(expand_rform_elem(k)[0][slot:slot+3]) in rel})) > 0})
            all_rels += [inst for inst in generate(fastsymb,rel,sp,3)]

    return all_rels

def get_all_checks(fweight,seamsize,bweight,pattern=None):    
    totlen=fweight+seamsize+bweight
    def isgood(myseam):
        bads={'ad','be','cf','da','eb','fc','df','fd','ef','fe','de','ed'}
        for i,j in zip(myseam[:-1],myseam[1:]):
            if ''.join([i,j]) in bads: return False
        return True
    
    def allseams(s):
        return [''.join(i) for i in product(alphabet, repeat=s) if isgood(i)]

    if not pattern in ["RR","PR","PP","RP"]:
        print("Error! bad sewmat configuration!")
        raise ValueError
    
    #get the sewing matrix and the possible slots

    if pattern in ["RR","RP"]:
        fkeys=rfspaces[fweight].keys()
    else:
        fkeys=pfspaces_flip[fweight].keys()    
        
    if pattern in ["RR","PR"]:
        bkeys=rbspaces[bweight].keys()
    else:
        bkeys=pbspaces_flip[bweight].keys()
    
    rspaces,pspaces,pslots=[],[],{'pair':{},'triple':{}}
    if pattern in ["RR","RP"]:rspaces.append("front")
    if pattern in ["RR","PR"]:rspaces.append("back")
    if pattern in ["PP","PR"]:pspaces.append("front")
    if pattern in ["PP","RP"]:pspaces.append("back")
    all_rels=[]
    
    pslots['pair']['front']=[fweight-1]
    pslots['triple']['front']=[fweight-2,fweight-1] if fweight > 1 else [fweight-1]
    pslots['pair']['back']=[seamsize]
    pslots['triple']['back']=[seamsize-1,seamsize] if seamsize > 1 else [seamsize]
    pslots['pair']['seam']=[k for k in range(0,seamsize-1)]
    pslots['triple']['seam']=[k for k in range(0,seamsize-2)]
    
    all_rels={}
    if seamsize > 0:
        all_rels['b_1']=[]
        all_rels['f_1']=[]
    if seamsize > 1:
        all_rels['b_2']=[]
        all_rels['f_2']=[]
    if seamsize > 1:
        all_rels['seam']=[]
        
    #expand the relevant f/b elem
    for sp in rspaces:
        print(f"getting {sp} pair rels")
        for rel in pair_rels:
            if sp == "back":
                fastsymb=fastRandomSampler({'@'.join([let,b]):0 for let in alphabet
                                            for b,blet in rbspaces[bweight].items() if ''.join([let,blet[0]]) in rel})          
                all_rels['b_1']+=[inst for inst in generate(fastsymb,rel,sp,2,0)]
            
            if sp == "front":
                fastsymb=fastRandomSampler({'@'.join([f,let]):0 for let in alphabet
                                        for f,flet in rfspaces[fweight].items() if ''.join([flet[-1],let]) in rel})
                all_rels['f_1']+= [inst for inst in generate(fastsymb,rel,sp,2,fweight-1)]
              
            
        print(f"getting {sp} triple rels")
        for rel in triple_rels:
            if sp == "front":
                fastsymb=fastRandomSampler({'@'.join([f,let]):0 for let in alphabet
                                            for f,flet in rfspaces[fweight].items() if ''.join([f[:-2],let]) in rel})
                
                all_rels['f_1'] += [inst for inst in generate(fastsymb,rel,sp,3,fweight-1)]
                if seamsize > 1:
                    fastsymb=fastRandomSampler({'@'.join([f,let]):0 for let in allseams(2)
                            for f,flet in rfspaces[fweight].items() if ''.join([f[-1],let]) in rel})
                all_rels['f_2'] += [inst for inst in generate(fastsymb,rel,sp,3,fweight-2)]
    
                
            if sp == "back":
                fastsymb=fastRandomSampler({'@'.join([let,b]):0 for let in alphabet
                                            for b,blet in rbspaces[bweight].items() if ''.join([let,blet[:2]]) in rel})                
                all_rels['b_1'] += [inst for inst in generate(fastsymb,rel,sp,3,0)]
                if seamsize > 1:
                    fastsymb=fastRandomSampler({'@'.join([let,b]):0 for let in allseams(2)
                                               for b,blet in rbspaces[bweight].items() if ''.join([let,blet[0]]) in rel})
                    all_rels['b_2'] += [inst for inst in generate(fastsymb,rel,sp,3,0)]

    #perm
    for sp in pspaces:
        print(f"getting {sp} permissive rels")
        if sp == "front":
            all_rels['f_1'] += pf_1l_sewchecks[fweight]
            if seamsize > 2:
                all_rels['f_2'] +=pf_2l_sewchecks[fweight]
        if sp == "back":
            all_rels['b_1'] += pb_1l_sewchecks[fweight]
            if seamsize > 2:
                all_rels['b_2'] += pb_2l_sewchecks[fweight]
    
    print("getting rels inside seam")
    for rel in pair_rels:
        fastsymb=fastRandomSampler({k:slotset for k in allseams(seamsize) 
                  if len(slotset := fastRandomSampler({slot for slot in pslots['pair']['seam'] if 
                                     ''.join(k[slot:slot+2]) in rel})) > 0})
        all_rels['seam'] += [inst for inst in gen_with_slots(fastsymb,rel,sp,2)]
   
    for rel in triple_rels:
        fastsymb=fastRandomSampler({k:slotset for k in allseams(seamsize) 
                  if len(slotset := fastRandomSampler({slot for slot in pslots['triple']['seam'] if 
                                     ''.join(k[slot:slot+3]) in rel})) > 0})
        all_rels['seam'] += [inst for inst in gen_with_slots(fastsymb,rel,sp,3)]

    flist1=['@'.join(i) for i in product(allseams(seamsize-1),bkeys)]
    flist2=['@'.join(i) for i in product(allseams(seamsize-2),bkeys)]
    blist1=['@'.join(i) for i in product(fkeys,allseams(seamsize-1))]
    blist2=['@'.join(i) for i in product(fkeys,allseams(seamsize-2))]
    edges=[i for i in product([fkeys,bkeys])]
    
    print("in aggregated 'c[i,a]' form, there are:")
    print(f"{len(all_rels['f_2'])+len(all_rels['f_1'])} checks across front seam")
    print(f"{len(all_rels['b_2'])+len(all_rels['b_1'])} checks across back seam")
    print(f"{len(all_rels['seam'])} checks inside seam")
    print("Gluing:")
    
    tot_rels=[]
    tot_rels += [{k+elem:v for k,v in rel.items()} for elem in flist1 for rel in all_rels['f_1']]
    tot_rels += [{k+elem:v for k,v in rel.items()} for elem in flist2 for rel in all_rels['f_2']]  
    tot_rels += [{elem+k:v for k,v in rel.items()} for elem in blist1 for rel in all_rels['b_1']]
    tot_rels += [{elem+k:v for k,v in rel.items()} for elem in blist2 for rel in all_rels['b_2']]
    tot_rels += [{'@'.join([e[0],k,e[1]]):v for k,v in rel.items()} for e in blist2 for rel in all_rels['seam']]
 
    print(f'there are {len(tot_rels)} checks!')
    return tot_rels

In [676]:
get_all_checks(2,3,3,"RR")

getting front pair rels
getting front triple rels
getting back pair rels
getting back triple rels
getting rels inside seam
('Fr_2_0', 'Br_3_0')
('Fr_2_0', 'Br_3_1')
('Fr_2_0', 'Br_3_2')
('Fr_2_0', 'Br_3_3')
('Fr_2_0', 'Br_3_4')
('Fr_2_0', 'Br_3_5')
('Fr_2_0', 'Br_3_6')
('Fr_2_0', 'Br_3_7')
('Fr_2_0', 'Br_3_8')
('Fr_2_0', 'Br_3_9')
('Fr_2_0', 'Br_3_10')
('Fr_2_0', 'Br_3_11')
('Fr_2_1', 'Br_3_0')
('Fr_2_1', 'Br_3_1')
('Fr_2_1', 'Br_3_2')
('Fr_2_1', 'Br_3_3')
('Fr_2_1', 'Br_3_4')
('Fr_2_1', 'Br_3_5')
('Fr_2_1', 'Br_3_6')
('Fr_2_1', 'Br_3_7')
('Fr_2_1', 'Br_3_8')
('Fr_2_1', 'Br_3_9')
('Fr_2_1', 'Br_3_10')
('Fr_2_1', 'Br_3_11')
('Fr_2_2', 'Br_3_0')
('Fr_2_2', 'Br_3_1')
('Fr_2_2', 'Br_3_2')
('Fr_2_2', 'Br_3_3')
('Fr_2_2', 'Br_3_4')
('Fr_2_2', 'Br_3_5')
('Fr_2_2', 'Br_3_6')
('Fr_2_2', 'Br_3_7')
('Fr_2_2', 'Br_3_8')
('Fr_2_2', 'Br_3_9')
('Fr_2_2', 'Br_3_10')
('Fr_2_2', 'Br_3_11')
('Fr_2_3', 'Br_3_0')
('Fr_2_3', 'Br_3_1')
('Fr_2_3', 'Br_3_2')
('Fr_2_3', 'Br_3_3')
('Fr_2_3', 'Br_3_4')
('Fr_2_3',

[{'Fr_2_6@faa@Br_3_0': 1},
 {'Fr_2_5@daa@Br_3_0': 1},
 {'Fr_2_0@daa@Br_3_0': 1},
 {'Fr_2_6@aaa@Br_3_0': 1},
 {'Fr_2_6@eaa@Br_3_0': 1},
 {'Fr_2_4@faa@Br_3_0': 1},
 {'Fr_2_3@eaa@Br_3_0': 1},
 {'Fr_2_8@eaa@Br_3_0': 1},
 {'Fr_2_4@baa@Br_3_0': 1},
 {'Fr_2_4@daa@Br_3_0': 1},
 {'Fr_2_8@caa@Br_3_0': 1,
  'Fr_2_8@aaa@Br_3_0': 1,
  'Fr_2_5@baa@Br_3_0': -1,
  'Fr_2_0@baa@Br_3_0': -1},
 {'Fr_2_2@eaa@Br_3_0': 1},
 {'Fr_2_1@baa@Br_3_0': 1,
  'Fr_2_1@aaa@Br_3_0': 1,
  'Fr_2_3@caa@Br_3_0': -1,
  'Fr_2_8@caa@Br_3_0': -1},
 {'Fr_2_5@caa@Br_3_0': 1,
  'Fr_2_5@baa@Br_3_0': 1,
  'Fr_2_7@aaa@Br_3_0': -1,
  'Fr_2_1@aaa@Br_3_0': -1},
 {'Fr_2_1@faa@Br_3_0': 1},
 {'Fr_2_7@faa@Br_3_0': 1},
 {'Fr_2_2@caa@Br_3_0': 1},
 {'Fr_2_2@daa@Br_3_0': 1},
 {'Fr_2_6@faa@Br_3_1': 1},
 {'Fr_2_5@daa@Br_3_1': 1},
 {'Fr_2_0@daa@Br_3_1': 1},
 {'Fr_2_6@aaa@Br_3_1': 1},
 {'Fr_2_6@eaa@Br_3_1': 1},
 {'Fr_2_4@faa@Br_3_1': 1},
 {'Fr_2_3@eaa@Br_3_1': 1},
 {'Fr_2_8@eaa@Br_3_1': 1},
 {'Fr_2_4@baa@Br_3_1': 1},
 {'Fr_2_4@daa@Br_3_1': 1},
 {'F

In [546]:
def get_relsymbs(partial_sewmat, w_f, w_b, loop):
    #GIVEN: a dict containing a subset of the sewing matrix in compressed form.
    # i.e. {f1_1}*{ab}*{B1_1}
    #within the seam, relations ONLY relate Mij to each other- no crossing of blocks!

    #pick a slot. check if there is a key with that the rel in the slot.
    #If there is, check off 
    
    relsymbs={}
    #Find all slots for pair rels that live purely in the seam
    for rel,i in enumerate(pair_rels):
        relsymbs[f'pair_{i}']={symbkey:slots for symbkey,slots in 
                              check_key_and_get_slots(partial_sewmat,loop,rel,None,"sewmat")}
    
    for rel,i in enumerate(triple_rels):
        relsymbs[f'triple_{i}']={symbkey:slots for symbkey,slots in 
                              check_key_and_get_slots(partial_sewmat,loop,rel,None,"sewmat")}
    
                
    
    
    
    
    
    
    
    
    
    #apply triple rels at all slots that live purely in the seam

    
    
    
    
    

    #for relsymb in relsymb_generator(nonlocal_rels, keydict):
    
    
                    
                    
                    
                    



In [547]:
print(pair_rels)

[{'df': 1}, {'dc': 1, 'db': -1, 'cd': -1, 'bd': 1, 'fb': 1, 'fa': -1, 'bf': -1, 'af': 1, 'ea': 1, 'ec': -1, 'ae': -1, 'ce': 1, 'bc': 2, 'cb': -2}, {'ad': 1}, {'da': 1}, {'de': 1}, {'ef': 1}, {'ec': 1, 'ea': -1, 'ce': -1, 'ae': 1, 'fa': 1, 'fb': -1, 'af': -1, 'bf': 1, 'db': 1, 'dc': -1, 'bd': -1, 'cd': 1, 'ac': 2, 'ca': -2}, {'be': 1}, {'eb': 1}, {'ed': 1}, {'bc': 1, 'ba': 1, 'cb': -1, 'ab': -1}, {'fe': 1}, {'cb': 1, 'ca': 1, 'bc': -1, 'ac': -1}, {'ac': 1, 'ab': 1, 'ca': -1, 'ba': -1}, {'fb': 1, 'fa': -1, 'bf': -1, 'af': 1, 'ea': 1, 'ec': -1, 'ae': -1, 'ce': 1, 'dc': 1, 'db': -1, 'cd': -1, 'bd': 1, 'ab': 2, 'ba': -2}, {'cf': 1}, {'fc': 1}, {'fd': 1}]


In [548]:
print(triple_rels)

[{'aab': 1, 'abb': 1, 'acb': 1}, {'aac': 1, 'acc': 1, 'abc': 1}, {'bba': 1, 'baa': 1, 'bca': 1}, {'bbc': 1, 'bcc': 1, 'bac': 1}, {'cca': 1, 'caa': 1, 'cba': 1}, {'ccb': 1, 'cbb': 1, 'cab': 1}]


In [549]:
for mykey in relsymb:
            for myslot in mykey:
                rel_keys=get_related(rel,mykey,myslot)

NameError: name 'relsymb' is not defined

In [550]:
def propagate_thru_brels(subsymb,weight):
    
#given the coefs for a set of individual fully-monomial terms in native letter representation:
#sort into those that live in indep bspace elements and those that don't
#use the relations to get the bspace elements in the orbits of the indeps
#there are XXX indeps remaining

#





Check: if F**bdd= F**dbd, then 



#given a monomial (assuming fully-restrictive representation):
#lookslike: {f_w_elem}{string}{b_w_elem}
def relterms_lookup()

#find the terms it is related to according to:
#at fboundary: integ0,integ1,integ2,triple1inFspace,triple2inFspace (or bspace)
#for each term in this set, do a lookup into the fspace/bspace and determine which
#of the indep basis elements we need to reconstruct it

#return the set of terms

IndentationError: expected an indented block after function definition on line 1 (3428622319.py, line 14)

In [551]:
#given a full symbol:

In [ ]:
"""
def check_key_and_get_slots(symb, rel):
    #for all keys in the symb, check whether they contain the desired substring in a valid slot. If so, store the slot.
    nletter=len(list(rel.keys())[0])
    my_slot= None
    for symbkey in symb:
        slots = {slot for key in rel for slot in find_all(symbkey,key)}
        if not slots: continue
        yield symbkey, slots

def relsymb_generator(rel, symb):
    return {symbkey: slots for symbkey, slots in check_key_and_get_slots(symb, rel)}
        
def get_related_keys(rel,key,slot):
    return set({(newkey := key[:slot]+relem+key[slot+2:]) for relem in rel if newkey != key})
"""

In [4]:
pair_rels=table_to_rels(pair_table)
triple_rels=table_to_rels(triple_table)
relpath='/home/garrett/LanceData_New/'
brels=get_brels(relpath)
frels=get_frels(relpath)

NameError: name 'table_to_rels' is not defined

In [ ]:
def get_all_related_keys(key,re):